In [12]:
import ollama

In [17]:
rstream = ollama.chat(
    model="llama3",
    messages=[
        {"role": "user", "content": "what comes after a"},
        {"role": "assistant", "content": "and after that"},
    ],
    stream = True
)

In [18]:
for msg in rstream:
    print(msg.message)

role='assistant' content='?\n\n' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content='In' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content=' the' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content=' English' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content=' alphabet' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content=',' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content=' "' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content='a' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content='"' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content=' is' thinking=None images=None tool_name=None tool_calls=None
role='assistant' content=' followed' thinking=None images=None tool_name=None tool_calls=None
role='assistant' con